In [ ]:
import numpy as np
import plotly.express as px
import scipy.stats as stats
from tqdm import tqdm

In [ ]:
SEED = 42
START = np.datetime64("2022-01-01T00:00:00")
DURATION = np.timedelta64(15, "s")
DELTA = np.timedelta64(10, "ms")

# Type of processes

## Wiener process

Z is a Wiener process if
1. $\Delta Z = \varepsilon \cdot\sqrt{\Delta{t}}$, where $\varepsilon \sim \Phi{(0,1)}$
2. $\Delta Z$ are independent for small ${\Delta{t}}$ intervals


## Generalized Wiener process
X is a generalized Wiener process if
$dX = a \,dt + b \,dZ$, where $Z$ is a Wiener process

In [ ]:
def getUnixTimeStamp(t: np.timedelta64):
    """Utility function to identify time units as integer indexes."""
    unit = str(t.dtype).split("[")[1][:-1]
    return t.astype(f"datetime64[{unit}]").astype("int")


def getIdentifier(t: np.timedelta64):
    """Map to smaller range."""
    nominator = np.timedelta64(1, "s") // np.timedelta64(1, "ns")
    return getUnixTimeStamp(t) % nominator

In [ ]:
def generateWiener(
    dt: np.timedelta64 = DELTA,
    z0=0,
    T: np.timedelta64 = DURATION,
    seed=SEED,
    disable_tqdm=False,
):
    """Generate Wiener process

    Each time creates new process. Doesn't change resolution of time.
    """

    np.random.seed(seed)
    Z = np.full(T // dt, z0, dtype=float)
    delta_t = dt / np.timedelta64(1, "s")
    for i in tqdm(
        range(1, Z.shape[0]), "Generating Wiener process", disable=disable_tqdm
    ):
        randomVal = stats.norm.rvs(0, 1)
        Z[i] = Z[i - 1] + randomVal * np.sqrt(delta_t)
    return Z


def generateGeneralWiener(
    a,
    b,
    dt: np.timedelta64 = DELTA,
    x0=0,
    T: np.timedelta64 = DURATION,
    seed=SEED,
    disable_tqdm=False,
):
    """Generate Wiener process

    Each time creates new process. Doesn't change resolution of time.
    """

    np.random.seed(seed)
    X = np.full(T // dt, x0, dtype=float)
    delta_t = dt / np.timedelta64(1, "s")
    print(X.shape)
    for i in tqdm(
        range(1, X.shape[0]), "Generating Wiener process", disable=disable_tqdm
    ):
        randomVal = stats.norm.rvs(0, 1)
        X[i] = a * delta_t + b * randomVal * np.sqrt(delta_t) + X[i - 1]
    return X

In [ ]:
px.line(generateGeneralWiener(3, 0.3))

In [ ]:
for num_ticks in [100 * DELTA, 10 * DELTA, DELTA]:
    Z = generateWiener(num_ticks)
    px.line(Z).show()